**Sommario**

- [Query di selezione con SQLite3](#query-di-selezione-con-sqlite3)
    - [`SELECT ... WHERE id = ?`: una riga della tabella in base alla chiave primaria](#select--where-id---una-riga-della-tabella-in-base-alla-chiave-primaria)
    - [`cursor.fetchall()`: tutti i risultati](#cursorfetchall-tutti-i-risultati)
    - [`cursor.fetchone()`: solo un risultato (alla volta)](#cursorfetchone-solo-un-risultato-alla-volta)
    - [`SELECT ... LIMIT 1`: solo il primo risultato](#select--limit-1-solo-il-primo-risultato)
    - [`SELECT ... LIMIT <n>`: numero di risultati limitato a una quantità specificata](#select--limit-n-numero-di-risultati-limitato-a-una-quantit%C3%A0-specificata)
    - [`SELECT ... WHERE <col> = ?`: risultati filtrati basandosi su parametri chiave-valore](#select--where-col---risultati-filtrati-basandosi-su-parametri-chiave-valore)
    - [`SELECT ... WHERE <col> <op> ?`: risultati filtrati basandosi su una condizione specificata](#select--where-col-op--risultati-filtrati-basandosi-su-una-condizione-specificata)
    - [`SELECT ... ORDER BY <col>`: risultati ordinati in base a uno o più criteri](#select--order-by-col-risultati-ordinati-in-base-a-uno-o-pi%C3%B9-criteri)
    - [`SELECT COUNT(*)`: numero di risultati](#select-count-numero-di-risultati)
    - [`SELECT DISTINCT`: risultati distinti senza duplicati](#select-distinct-risultati-distinti-senza-duplicati)
    - [`SELECT ... INNER JOIN ... ON ...`: INNER JOIN tra tabelle](#select--inner-join--on--inner-join-tra-tabelle)
    - [`SELECT ... LEFT JOIN ... ON ...` : OUTER JOIN tra tabelle](#select--left-join--on---outer-join-tra-tabelle)

# Query di selezione con SQLite3

Dato che l'operazione di READ (R di CRUD) prevede molte possibili query di selezione, vediamo le principali operazioni di lettura dal database in questo notebook, come argomento a parte.

Le restanti operazioni del pattern CRUD, CREATE, UPDATE e DELETE le trovate nell'altro notebook, [sqlite3_CRUD.ipynb](sqlite3_CRUD.ipynb)

### `SELECT ... WHERE id = ?`: una riga della tabella in base alla chiave primaria

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("SELECT * FROM users WHERE id = ?", (1,))
user = cursor.fetchone()

conn.close()
```

OUTPUT: Restituisce la riga che ha la chiave primaria uguale a 1 sotto forma di tupla. Se non esiste, restituisce `None`.

### `cursor.fetchall()`: tutti i risultati

Se eseguite una query e povi volete ottenere tutte le righe trovate in una lista di tuple. potete usare il metodo `.fetchall()` del cursore:

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("SELECT * FROM users")  # Una qualunque query
users = cursor.fetchall()

conn.close()
```

OUTPUT: Restituisce tutte le righe della tabella `users` sotto forma di una lista di tuple.

### `cursor.fetchone()`: solo un risultato (alla volta)

Se eseguite una query e povi volete ottenere un solo record alla volta partendo dal primo, potete usare il metodo `.fetchone()` del cursore:

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("SELECT * FROM users")  # Una qualunque query
first_user = cursor.fetchone()

conn.close()
```

OUTPUT: Restituisce la prima riga della tabella `users` sotto forma di tupla. Se la tabella è vuota, restituisce `None`. Se `.fetchone()` viene invocato di nuovo, restituisce la seconda riga e così via ogni volta che viene invocato.

> **ATTENZIONE**: Se eseguite un ciclo `for` su un cursore dopo che ha eseguito una query, è come se invocaste `.fetchone()` a ogni ciclo e otterrete una riga alla volta:
> ```python
> import sqlite3
> 
> conn = sqlite3.connect('db.sqlite3')
> cursor = conn.cursor()
> 
> for user in cursor.execute("SELECT * FROM users")  # Una qualunque query
>     print(user)  # Stampa una riga alla volta sotto forma di tupla
> 
> conn.close()
> ```

### `SELECT ... LIMIT 1`: solo il primo risultato

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("SELECT * FROM users LIMIT 1")
limited_users = cursor.fetchone()

conn.close()
```

OUTPUT: Restituisce la prima riga della tabella `users` sotto forma di tupla. Se la tabella è vuota, restituisce `None`.

### `SELECT ... LIMIT <n>`: numero di risultati limitato a una quantità specificata

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("SELECT * FROM users LIMIT 10")
limited_users = cursor.fetchall()

conn.close()
```

OUTPUT: Restituisce le prime 10 righe della tabella `users` sotto forma una lista di tuple.

### `SELECT ... WHERE <col> = ?`: risultati filtrati basandosi su parametri chiave-valore

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("SELECT * FROM users WHERE name = ?", ('Alice',))
filtered_users = cursor.fetchall()

conn.close()
```

OUTPUT: Restituisce le righe che hanno il nome 'Alice' sotto forma di una lista di tuple.

### `SELECT ... WHERE <col> <op> ?`: risultati filtrati basandosi su una condizione specificata

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute('SELECT * FROM users WHERE age > ?', (18, ))
filtered_users = cursor.fetchall()

conn.close()
```

OUTPUT: Restituisce le righe che soddisfano la condizione specificata (in questo caso, utenti con età superiore a 18) sotto forma di una lista di tuple.

### `SELECT ... ORDER BY <col>`: risultati ordinati in base a uno o più criteri

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("SELECT * FROM users ORDER BY name")
asc_sorted_users = cursor.fetchall()

cursor.execute("SELECT * FROM users ORDER BY name DESC")
desc_sorted_users = cursor.fetchall()

conn.close()
```

OUTPUT: Restituisce le righe della tabella `users` ordinate per nome in ordine crescente oppure decrescente se viene aggiunto `DESC` alla query. I risultati sono sotto forma di una lista di tuple.

### `SELECT COUNT(*)`: numero di risultati

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute('SELECT COUNT(*) FROM users WHERE age > 18')
user_count = cursor.fetchone()[0]  # Accede al primo elemento della tupla

conn.close()
```

OUTPUT: Restituisce un numero intero rappresentante il conteggio degli utenti con età superiore a 18.

### `SELECT DISTINCT`: risultati distinti senza duplicati

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("SELECT DISTINCT name FROM users")
distinct_names = cursor.fetchall()

conn.close()
```

OUTPUT: Restituisce una lista di tuple con i nomi distinti degli utenti.

> **NOTA**: In casi come questo in cui il risultato è una lista di tuple con un solo elemento, per ottenere una lista di stringhe, puoi estrarre il primo elemento di ogni tupla con una list comprehension:
> ```python
>     names_list = [res[0] for res in distinct_names]
> ```

### `SELECT ... INNER JOIN ... ON ...`: INNER JOIN tra tabelle

Sintassi generale:

```sql
SELECT <tab1>.<col>, [<tab2>.<col>]
FROM <tab1>
INNER JOIN <tab2> ON <tab1>.id = <tab2>.<tab1_id>
```

Esempio Python:

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute("""
    SELECT users.*, posts.* 
    FROM users 
    INNER JOIN posts ON users.id = posts.user_id
""")
user_posts = cursor.fetchall()

conn.close()
```

OUTPUT: Restituisce le righe di `users` e `posts` unite tramite una `INNER JOIN`, secondo la condizione specificata, sotto forma di lista di tuple. Ciascuna tula contiene tutte le colonne sia di `users` sia di `posts`.

> **NOTA**: La `INNER JOIN` mostra solo le righe che hanno una reciproca corrispondenza in entrambe le tabelle.

### `SELECT ... LEFT JOIN ... ON ...` : OUTER JOIN tra tabelle

Sintassi generale:

```sql
SELECT <tab1>.<col>, [<tab2>.<col>]
FROM <tab1>
LEFT JOIN <tab2> ON <tab1>.id = <tab2>.<tab1_id>
```

Esempio Python:

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

# Eseguire un LEFT OUTER JOIN tra users e posts
cursor.execute("""
    SELECT users.*, posts.*
    FROM users
    LEFT JOIN posts ON users.id = posts.user_id
""")
user_posts = cursor.fetchall()

conn.close()
```

OUTPUT: Restituisce le righe di `users` e `posts` unite tramite una `INNER JOIN`, secondo la condizione specificata, sotto forma di lista di tuple. Ciascuna tula contiene tutte le colonne sia di `users` sia di `posts`. Se un utente non ha post corrispondenti, le colonne della tabella `posts` saranno `NULL`.

> **NOTA BENE**: La `LEFT OUTER JOIN` mostra tutte le righe della tabella di sinistra (`users`), e le righe corrispondenti della tabella di destra (`posts`). Se non ci sono corrispondenze, le colonne della tabella di destra saranno `NULL`.